In [ ]:
!pip install -U sentence-transformers

In [ ]:
from sentence_transformers import SentenceTransformer, InputExample, losses
import pandas as pd
from sentence_transformers import SentenceTransformer, InputExample
from torch.utils.data import DataLoader
from sentence_transformers import SentenceTransformer, util

import requests
import json

sim_model_name="Sakil/sentence_similarity_semantic_search"
sim_model = SentenceTransformer(sim_model_name)

In [ ]:
# Append the name of the rule to the contents of the rule description
# Create a lists of: rules, subreddits, and number of rules per subreddit
def sentence_in(reddits):
    sub_rules = []
    sub_out = []
    num_rules = []
    for sub, rules in reddits.items():
        sub_out.append(sub)
        i = 0
        for rule in rules.values():
            sub_rules.append(rule["short_name"] + " " + rule["description"])
            i = i + 1
        num_rules.append(i)
    return sub_rules, sub_out, num_rules

def sub_rules():
    f_open = open("rightrules.json")
    reddits = json.load(f_open)
    data_all = sentence_in(reddits)
    return data_all

data, subs_out, num_rules = sub_rules()

In [ ]:
#https://huggingface.co/Sakil/sentence_similarity_semantic_search
#Encode all sentences
embeddings = sim_model.encode(data)

#Compute cosine similarity between all pairs
cos_sim = util.cos_sim(embeddings, embeddings)

#Add all pairs to a list with their cosine similarity score
all_sentence_combinations = []

for i in range(len(cos_sim)-1):
    for j in range(i+1, len(cos_sim)):
        # check for cosine similarity of over 0.9
        if cos_sim[i][j] > 0.9:
            # find the subreddits the rules are associated with
            xx = 0
            ii = 0
            jj = 0
            sub_i = " "
            sub_j = " "
            for w in range(len(num_rules)):
                if i >= xx - 1 and i < xx + num_rules[w]:
                    sub_i = subs_out[ii]
                if j >= xx - 1 and j < xx + num_rules[w]:
                    sub_j = subs_out[jj]
                ii += 1
                jj += 1
                xx += num_rules[w]
            if sub_i != sub_j:
                all_sentence_combinations.append([cos_sim[i][j], data[i], data[j], sub_i, sub_j])

In [ ]:
# make a plot of of number of rules that overlap based on the cosine similarity threshold

import matplotlib.pyplot as plt
import numpy as np

graph_1 = []
graph_count = []
#counter = [0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.85, 0.9, 0.95, 1]
counter = [0.7, 0.75, 0.8, 0.85, 0.9, 0.95, 1]
for n in counter:
    tab = 0
    for i in range(len(cos_sim)-1):
        for j in range(i+1, len(cos_sim)):
            if cos_sim[i][j] > n:
                tab += 1
    graph_count.append(tab)

xpoints = np.array(counter)
ypoints = np.array(graph_count)


plt.plot(xpoints, ypoints)
plt.show()

In [ ]:
all_sentence_combinations_over_no = []

# create a new list all_sentence_combinations_over_no where subreddits are a pair
for row in all_sentence_combinations:
    row_new = []
    row_new.append(row[0])
    row_new.append(row[1])
    row_new.append(row[2])
    row_new.append([row[3], row[4]])
    all_sentence_combinations_over_no.append(row_new)

# sort based on the pair of subreddits
all_sentence_combinations_over_no = sorted(all_sentence_combinations_over_no, key=lambda x: x[3], reverse=True)

In [ ]:
all_sentence_combinations_out = sorted(all_sentence_combinations, key=lambda x: x[0], reverse=True)

In [ ]:
# overlap_A: a list of all of the subreddits with overlapping rules above 0.9 not connected\
# to the community the rule has overlap with
# overlap_both: List of each instance where a rule has overlap between communities taking\
# into account the subreddits of both rules
overlap_A = []
overlap_both = []
for row in all_sentence_combinations_out:
    overlap_A.append(row[3])
    overlap_A.append(row[4])
    overlap_both.append([row[3], row[4]])

In [ ]:
# Right leaning communities had a number of fully overlapping rules
# overall: list of subreddits with fully copied rules
overall = []
# the_same_comment: a dictionary mapping comments to the subreddits that
# have the same rule
the_same_comment = {}
for line in all_sentence_combinations_out:
    # line[0] is cosine similarity
    if line[0] >= 1:
        # line[1] is the rule
        if line[1] not in the_same_comment:
            the_same_comment[line[1]] = []
        # append both subreddits to the dictionary as well as overall
        the_same_comment[line[1]].append(line[3])
        the_same_comment[line[1]].append(line[4])
        overall.append(line[3])
        overall.append(line[4])

In [ ]:
for val in the_same_comment:
    # take the set of all the subreddits with the same rule
    the_same_comment[val] = list(set(the_same_comment[val]))

In [ ]:
for val in the_same_comment:
    print("Num subreddits:", len(the_same_comment[val]), " Rule:", val)
    print(the_same_comment[val])
print("Subreddits with >=1 fully copied rule: ", len(list(set(overall))))


In [ ]:
import pandas as pd

count = pd.Series(overlap_A).value_counts()
print("Right Leaning Subreddits With Most Overlapping Rules:")
print(count)

In [ ]:
count = pd.Series(overlap_both).value_counts()
print("Right Leaning Subreddits That Overlap With Eachother")
print(count)

Emotion

In [ ]:
from transformers import pipeline
# https://huggingface.co/SamLowe/roberta-base-go_emotions
classifier_emo = pipeline(task="text-classification", model="SamLowe/roberta-base-go_emotions", top_k=None)

In [ ]:
model_outputs = classifier_emo(data)

In [ ]:
emotions = []
high_sub = []
i = 0
for line in model_outputs:
    for emo in line:
        if emo['score'] > 0.5:
            emotions.append(emo['label'])
            if emo['label'] != 'neutral':
                high_sub.append([i, emo['label'], emo['score']])
    i += 1

In [ ]:
# print emotions with values of over 0.5
count_sentiment = pd.Series(emotions).value_counts()
print("Emotion over 0.5 Count")
print(count_sentiment)

In [ ]:
# print non neutral emotinos
print(high_sub)

In [ ]:
emotions = []
for line in model_outputs:
    for emo in line:
        if emo['score'] > 0.75:
            print(emo)
            emotions.append(emo['label'])

In [ ]:
# print emotions with values of over 0.5
count_sentiment = pd.Series(emotions).value_counts()
print("Emotion over 0.75 Count")
print(count_sentiment)

Sentiment

In [ ]:
from transformers import pipeline
# https://huggingface.co/blog/sentiment-analysis-python
sentiment_pipeline = pipeline("sentiment-analysis")

In [ ]:
cat = sentiment_pipeline(data)

In [ ]:
print(cat)
print(len(data))

In [ ]:
pos = 0
neg = 0
under = 0
for line in cat:
  under += 1
  if line['label'] == 'POSITIVE':
      pos += 1
  if line['label'] == 'NEGATIVE':
      neg += 1

print("pos: ", pos)
print("neg: ", neg)

In [ ]:
print("Postive Rules: ", pos)
print("Negative Rules ", neg)
print("Percentage Negative: ", neg/(under) * 100, "%")